In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
def process_trip(single_trip):

    arrivals = single_trip[['station_char']].drop_duplicates()
    arrival_max = single_trip[single_trip['timint']!=0]
    arrival_max = arrival_max.sort_values(by = 'request_date', ascending = False)
    arrival_max = arrival_max.drop_duplicates(subset = ['station_char'])[['station_char', 'estimated_arrival']]

    arrivals_at_station = single_trip[single_trip['timint']==0]
    arrivals_at_station = arrivals_at_station.sort_values(by = 'estimated_arrival')
    arrivals_at_station  = arrivals_at_station.drop_duplicates(subset = ['station_char'])[['station_char',
                                                                                           'estimated_arrival']]
    arrivals = arrivals.merge(arrival_max, how = 'left')

    arrivals_at_station = arrivals_at_station.rename(columns = {'estimated_arrival':'at_station_time'})
    arrivals = arrivals.merge(arrivals_at_station, how = 'left')
    arrivals['arrival'] = True

    arrivals['estimated_arrival'] = np.where(arrivals['estimated_arrival'].isna(), 
                                             arrivals['at_station_time'], arrivals['estimated_arrival'])

    arrivals['estimated_arrival'] = np.where(arrivals['estimated_arrival']>arrivals['at_station_time'], 
                                             arrivals['at_station_time'], arrivals['estimated_arrival'])

    arrivals = arrivals.sort_values(by = 'estimated_arrival').reset_index(drop = True)
    
    departures = single_trip[['station_char']].drop_duplicates()
    departures_max = single_trip[single_trip['timint'] == 0]
    departures_max = departures_max.sort_values(by = 'estimated_arrival', ascending = False)
    departures_max = departures_max.drop_duplicates(subset = ['station_char'])[['station_char', 'estimated_arrival']]

    departures = departures.merge(departures_max, how = 'left')
    departures['departure'] = True

    departures = departures.merge(arrivals.rename(columns = {'estimated_arrival':'before_station_time'}))

    departures['estimated_arrival'] = np.where(departures['estimated_arrival'].isna(), departures['before_station_time'], departures['estimated_arrival'])
    departures = departures[['station_char', 'estimated_arrival', 'departure']]
    departures = departures.sort_values(by = 'estimated_arrival').reset_index(drop = True)
    arrivals['previous_depart'] = departures['estimated_arrival'].shift(1)


    arrivals['estimated_arrival'] = np.where(arrivals['estimated_arrival'] < arrivals['previous_depart'], 
                                         arrivals['at_station_time'], arrivals['estimated_arrival'])
    
    arrivals = arrivals[['station_char', 'estimated_arrival', 'arrival']]
    
    single_trip = single_trip.merge(arrivals, how = 'left')
    single_trip = single_trip.merge(departures, how = 'left')

    single_trip = single_trip.sort_values(by = 'estimated_arrival')
    
    single_trip = single_trip[['station_char', 'subwayline', 'timint', 'traindirection', 'trip','trainid',
                 'stationid', 'estimated_arrival', 'arrival', 'departure']]
    
    return single_trip[~single_trip['departure'].isna()].reset_index(drop = True), single_trip[~single_trip['arrival'].isna()].reset_index(drop = True)

In [3]:
df = pd.read_csv('cleaned_subway/raw_subway_2019-11-28_AM.csv')

In [4]:
df['request_date'] = pd.to_datetime(df['request_date'])

In [5]:
df = df.sort_values(by = 'timint')

In [6]:
df['platformid'] = df['station_char'].str[3]

In [7]:
df

,requestid,id,station_char,subwayline,timint,traindirection,trainid,train_message,stationid,lineid,create_date,pollid,request_date,date,hour,dow,period,platformid
96041,2305871,34246572163,YIE1,SHEP,0.000000,East,463,AtStation,64,4,2019-11-28 08:25:51,32685,2019-11-28 08:25:51.333503-05:00,2019-11-28,8,3,AM,1
70449,2284190,34250776188,DNW1,BD,0.000000,East,238,AtStation,41,2,2019-11-28 11:47:46,32390,2019-11-28 11:47:43.252615-05:00,2019-11-28,11,3,AM,1
108705,2317542,34249245226,DAV1,YUS,0.000000,North,23,AtStation,26,1,2019-11-28 10:33:44,32845,2019-11-28 10:33:43.807289-05:00,2019-11-28,10,3,AM,1
130034,2331258,34250039928,STP1,YUS,0.000000,North,142,AtStation,13,1,2019-11-28 11:11:43,33032,2019-11-28 11:11:43.844225-05:00,2019-11-28,11,3,AM,1
20763,2253499,34248521900,YNG2,BD,0.000000,West,209,AtStation,22,1,2019-11-28 09:58:44,31970,2019-11-28 09:58:43.613983-05:00,2019-11-28,9,3,AM,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131973,2332904,34246290251,PVL2,YUS,23.830480,South,101,Arriving,78,1,2019-11-28 08:11:46,33054,2019-11-28 08:11:43.647875-05:00,2019-11-28,8,3,AM,2
43569,2267264,34246374797,YUN2,YUS,24.054304,South,102,Arriving,77,1,2019-11-28 08:15:46,32158,2019-11-28 08:15:43.330515-05:00,2019-11-28,8,3,AM,2
80650,2293189,34246332696,HWY2,YUS,24.077273,South,101,Arriving,79,1,2019-11-28 08:13:46,32513,2019-11-28 08:13:43.803495-05:00,2019-11-28,8,3,AM,2
27515,2257777,34246271179,PVL2,YUS,25.284794,South,101,Arriving,78,1,2019-11-28 08:10:55,32029,2019-11-28 08:10:51.335830-05:00,2019-11-28,8,3,AM,2


# Filtering for only the most recent data in a request

Since each request has multiple train arrival times.

We're filtering on `station_char` and `requestid`. This might cause an issue with terminus, but we'll deal with it later.

In [8]:
df_newest = df.drop_duplicates(subset = ['requestid', 'station_char'])

In [9]:
train_list = []
for line in list(df_newest['subwayline'].drop_duplicates()):
    df_line = df_newest[df_newest['subwayline'] == line]
    
    train_ids = df_line['trainid'].drop_duplicates()
    
    for train_id in train_ids:
        
        single_train = df_line[df_line['trainid'] == train_id]
        
        single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')

        single_train = single_train.sort_values(by = 'request_date')

        single_train['trip'] = (single_train['traindirection'] != single_train['traindirection'].shift(1)).cumsum() - 1
        
        trip_num = single_train['trip'].max()
        
        arrival_list = []
        departure_list = []
        for i in range(trip_num + 1):
            if i == 0:
                current_departures, current_arrivals = process_trip(single_train[single_train['trip'] == i])

            if i < trip_num:
                next_departures, next_arrivals = process_trip(single_train[single_train['trip'] == i + 1])

            arrival_limit = next_arrivals.head(1)[['estimated_arrival']].iloc[0,0] - datetime.timedelta(seconds = 45)

            if len(current_arrivals['station_char']) == 1:
                pass

            elif i < trip_num:

                current_departures = current_departures[current_departures['estimated_arrival'] < arrival_limit]
                current_arrivals = current_arrivals[current_arrivals['estimated_arrival'] < arrival_limit]
            else:
                pass

            arrival_list.append(current_arrivals.copy())
            departure_list.append(current_departures.copy())

            current_arrivals = next_arrivals
            current_departures = next_departures


        arrival_trip = pd.concat(arrival_list)
        departure_trip = pd.concat(departure_list)        

        if arrival_trip['estimated_arrival'].max().hour < 11:
            if arrival_trip['subwayline'].drop_duplicates().iloc[0] == 'BD':
                cutoff = arrival_trip[arrival_trip['stationid'] == 55]['estimated_arrival'].max()

            elif (arrival_trip['subwayline'].drop_duplicates().iloc[0] == 'YUS') & (arrival_trip.tail(1)['stationid'].iloc[0] == 80):
                cutoff = arrival_trip[arrival_trip['stationid'] == 80]['estimated_arrival'].max()

            elif arrival_trip['subwayline'].drop_duplicates().iloc[0] == 'YUS':
                cutoff = arrival_trip[arrival_trip['stationid'] == 1]['estimated_arrival'].max()
            else:
                cutoff = arrival_trip['estimated_arrival'].max()
        else:
            cutoff = arrival_trip['estimated_arrival'].max()

        arrival_trip = arrival_trip[arrival_trip['estimated_arrival'] <= cutoff]
        departure_trip = departure_trip[departure_trip['estimated_arrival'] <= cutoff]  

        arrival_trip = arrival_trip.reset_index(drop = True).reset_index()

        departure_trip = departure_trip.reset_index(drop = True).reset_index()
        departure_trip['index'] = departure_trip['index'] + 1

        departure_trip = departure_trip.rename(columns = {'station_char':'dep_stn_char', 'estimated_arrival':'dep_time'})
        arrival_trip = arrival_trip.rename(columns = {'station_char':'arr_stn_char', 'estimated_arrival':'arr_time'})

        train = departure_trip.merge(arrival_trip[['index', 'arr_stn_char', 'arr_time']]).rename(
            columns = {'index':'sequence'})[['sequence','dep_stn_char', 'arr_stn_char', 'subwayline', 'trainid',
                                                 'trip', 'dep_time', 'arr_time']]
        
        
        train_list.append(train)


/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_9645/1506887821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_train['estimated_arrival'] = single_train['request_date'] + pd.to_timedelta(single_train['timint'], 'm')
/var/folders/zw/x46df3ms6r5391ly

In [10]:
train_df = pd.concat(train_list)

In [11]:
train_df['cost'] = (train_df['arr_time'] - train_df['dep_time']).dt.seconds/60

In [12]:
train_df.sort_values(by = 'cost', ascending = False).head(50)

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
18,19,SGU1,SGU1,YUS,108,2,2019-11-28 09:30:51.366777-05:00,2019-11-28 09:30:43.686963-05:00,1439.866667
17,18,SPA1,SPA1,YUS,108,2,2019-11-28 09:28:51.367157-05:00,2019-11-28 09:28:43.901357-05:00,1439.866667
38,39,SHP1,SHP1,YUS,108,2,2019-11-28 10:05:51.101114-05:00,2019-11-28 10:05:43.694809-05:00,1439.866667
10,11,SPA1,SPA1,YUS,62,1,2019-11-28 09:01:51.353677-05:00,2019-11-28 09:01:43.767019-05:00,1439.866667
26,27,OML2,RYK2,BD,241,0,2019-11-28 07:58:43.539957-05:00,2019-11-28 07:57:43.597888-05:00,1439.000000
60,61,QPK1,MUS1,YUS,171,2,2019-11-28 09:15:43.386011-05:00,2019-11-28 09:14:43.547731-05:00,1439.000000
11,12,SGU1,SGU1,YUS,62,1,2019-11-28 09:03:51.354330-05:00,2019-11-28 09:02:51.354445-05:00,1439.000000
30,31,BLO1,BLO1,YUS,108,2,2019-11-28 09:48:51.109399-05:00,2019-11-28 09:47:51.101384-05:00,1438.983333
62,63,OSG1,STP1,YUS,171,2,2019-11-28 09:18:43.695999-05:00,2019-11-28 09:17:43.634076-05:00,1438.983333
57,58,SPA1,DUP1,YUS,171,2,2019-11-28 09:10:51.356299-05:00,2019-11-28 09:09:43.229177-05:00,1438.850000


In [13]:
train_df

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
0,1,BYV1,BSS1,SHEP,463,0,2019-11-28 07:01:51.417028-05:00,2019-11-28 07:03:45.173927240-05:00,1.883333
1,2,BSS1,LES1,SHEP,463,0,2019-11-28 07:03:45.173927240-05:00,2019-11-28 07:04:51.416831-05:00,1.100000
2,3,LES1,DML2,SHEP,463,0,2019-11-28 07:04:51.416831-05:00,2019-11-28 07:07:51.419204-05:00,3.000000
3,4,DML2,LES2,SHEP,463,1,2019-11-28 07:10:51.422222-05:00,2019-11-28 07:12:51.427952-05:00,2.000000
4,5,LES2,BSS2,SHEP,463,1,2019-11-28 07:12:51.427952-05:00,2019-11-28 07:14:46.391402980-05:00,1.900000
...,...,...,...,...,...,...,...,...,...
3,4,FIW1,DNP1,YUS,52,0,2019-11-28 10:58:43.494615-05:00,2019-11-28 11:01:43.995897-05:00,3.000000
4,5,DNP1,SHW1,YUS,52,0,2019-11-28 11:01:43.995897-05:00,2019-11-28 11:04:35.538133500-05:00,2.850000
0,1,WIL1,SHW2,YUS,199,0,2019-11-28 11:55:27.556311960-05:00,2019-11-28 11:56:53.243854680-05:00,1.416667
1,2,SHW2,DNP2,YUS,199,1,2019-11-28 11:56:53.243854680-05:00,2019-11-28 11:59:26.011504840-05:00,2.533333


In [14]:
train_df[(train_df['cost'] > 0.4) & (train_df['cost'] < 15)]

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
0,1,BYV1,BSS1,SHEP,463,0,2019-11-28 07:01:51.417028-05:00,2019-11-28 07:03:45.173927240-05:00,1.883333
1,2,BSS1,LES1,SHEP,463,0,2019-11-28 07:03:45.173927240-05:00,2019-11-28 07:04:51.416831-05:00,1.100000
2,3,LES1,DML2,SHEP,463,0,2019-11-28 07:04:51.416831-05:00,2019-11-28 07:07:51.419204-05:00,3.000000
3,4,DML2,LES2,SHEP,463,1,2019-11-28 07:10:51.422222-05:00,2019-11-28 07:12:51.427952-05:00,2.000000
4,5,LES2,BSS2,SHEP,463,1,2019-11-28 07:12:51.427952-05:00,2019-11-28 07:14:46.391402980-05:00,1.900000
...,...,...,...,...,...,...,...,...,...
3,4,FIW1,DNP1,YUS,52,0,2019-11-28 10:58:43.494615-05:00,2019-11-28 11:01:43.995897-05:00,3.000000
4,5,DNP1,SHW1,YUS,52,0,2019-11-28 11:01:43.995897-05:00,2019-11-28 11:04:35.538133500-05:00,2.850000
0,1,WIL1,SHW2,YUS,199,0,2019-11-28 11:55:27.556311960-05:00,2019-11-28 11:56:53.243854680-05:00,1.416667
1,2,SHW2,DNP2,YUS,199,1,2019-11-28 11:56:53.243854680-05:00,2019-11-28 11:59:26.011504840-05:00,2.533333


In [15]:
train_df[(train_df['cost'] < 0.4)].sort_values(by = 'cost')

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
7,8,YIE2,YIE2,SHEP,463,2,2019-11-28 07:19:21.630879300-05:00,2019-11-28 07:19:21.630879300-05:00,0.000000
98,99,KEN1,KEN2,BD,219,6,2019-11-28 10:01:15.610867580-05:00,2019-11-28 10:01:15.610867580-05:00,0.000000
159,160,KEN2,KEN2,BD,219,7,2019-11-28 11:43:31.455018980-05:00,2019-11-28 11:43:31.455018980-05:00,0.000000
15,16,KIP2,KIP1,BD,226,1,2019-11-28 07:33:59.230275859-05:00,2019-11-28 07:33:59.230275859-05:00,0.000000
82,83,KIP1,KIP2,BD,232,2,2019-11-28 09:28:20.244641160-05:00,2019-11-28 09:28:20.244641160-05:00,0.000000
...,...,...,...,...,...,...,...,...,...
75,76,STA2,DUP2,YUS,151,2,2019-11-28 09:44:23.789823900-05:00,2019-11-28 09:44:43.719085-05:00,0.316667
87,88,GCN2,STA2,YUS,158,2,2019-11-28 10:07:50.962539260-05:00,2019-11-28 10:08:11.148869620-05:00,0.333333
75,76,BSS1,LES1,SHEP,461,15,2019-11-28 09:48:23.299727900-05:00,2019-11-28 09:48:44.007759700-05:00,0.333333
58,59,EGL2,YUN2,YUS,145,4,2019-11-28 09:14:43.553837-05:00,2019-11-28 09:15:05.079749700-05:00,0.350000


In [16]:
train_df[(train_df['dep_stn_char'].str[0:3] == train_df['arr_stn_char'].str[0:3])]

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
7,8,YIE2,YIE2,SHEP,463,2,2019-11-28 07:19:21.630879300-05:00,2019-11-28 07:19:21.630879300-05:00,0.000000
12,13,DML1,DML1,SHEP,463,3,2019-11-28 07:29:16.561617760-05:00,2019-11-28 07:29:16.561617760-05:00,0.000000
21,22,DML2,DML1,SHEP,463,4,2019-11-28 07:50:24.412125120-05:00,2019-11-28 07:50:24.412125120-05:00,0.000000
26,27,YIE2,YIE2,SHEP,463,6,2019-11-28 08:02:21.518839300-05:00,2019-11-28 08:02:21.518839300-05:00,0.000000
31,32,DML2,DML1,SHEP,463,6,2019-11-28 08:12:24.421907120-05:00,2019-11-28 08:12:24.421907120-05:00,0.000000
...,...,...,...,...,...,...,...,...,...
34,35,DAV1,DAV1,YUS,108,2,2019-11-28 09:54:43.462160-05:00,2019-11-28 09:54:43.462160-05:00,0.000000
35,36,EGL1,EGL1,YUS,108,2,2019-11-28 09:56:43.797668-05:00,2019-11-28 09:56:43.797668-05:00,0.000000
36,37,LAW1,LAW1,YUS,108,2,2019-11-28 09:59:43.716490-05:00,2019-11-28 09:59:43.716490-05:00,0.000000
37,38,YKM1,YKM1,YUS,108,2,2019-11-28 10:02:43.423479-05:00,2019-11-28 10:02:43.423479-05:00,0.000000


In [17]:
train_df[(train_df['dep_stn_char'].str[0:3] != train_df['arr_stn_char'].str[0:3]) & (train_df['cost'] < 0.4 )].reset_index(drop = True)

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
0,54,BSS2,BYV2,SHEP,463,11,2019-11-28 09:05:46.313604980-05:00,2019-11-28 09:05:51.348379-05:00,0.083333
1,36,LES2,BSS2,SHEP,464,8,2019-11-28 08:25:20.964934920-05:00,2019-11-28 08:25:28.536897680-05:00,0.116667
2,55,BYV1,BSS1,SHEP,461,11,2019-11-28 09:03:08.058430400-05:00,2019-11-28 09:03:08.984957920-05:00,0.000000
3,76,BSS1,LES1,SHEP,461,15,2019-11-28 09:48:23.299727900-05:00,2019-11-28 09:48:44.007759700-05:00,0.333333
4,56,RUN2,RYK2,BD,213,2,2019-11-28 08:38:31.581716720-05:00,2019-11-28 08:38:43.977257-05:00,0.200000
5,5,ISL2,DUF2,BD,227,0,2019-11-28 07:25:06.623537-05:00,2019-11-28 07:25:24.838029620-05:00,0.300000
6,71,MUS2,KNG2,YUS,151,2,2019-11-28 09:39:43.558247-05:00,2019-11-28 09:39:56.879535740-05:00,0.216667
7,76,STA2,DUP2,YUS,151,2,2019-11-28 09:44:23.789823900-05:00,2019-11-28 09:44:43.719085-05:00,0.316667
8,81,BLO2,DUP2,YUS,158,2,2019-11-28 09:57:40.222264380-05:00,2019-11-28 09:57:46.118908740-05:00,0.083333
9,88,GCN2,STA2,YUS,158,2,2019-11-28 10:07:50.962539260-05:00,2019-11-28 10:08:11.148869620-05:00,0.333333


In [18]:
train_df[(train_df['cost'] > 1000)].reset_index(drop = True)

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
0,1,BYV1,YIE1,SHEP,413,0,2019-11-28 12:02:16.729921740-05:00,2019-11-28 11:59:59.093568820-05:00,1437.700000
1,27,OML2,RYK2,BD,241,0,2019-11-28 07:58:43.539957-05:00,2019-11-28 07:57:43.597888-05:00,1439.000000
2,10,GWD1,WDB1,BD,201,0,2019-11-28 07:19:51.426193-05:00,2019-11-28 07:17:51.429492-05:00,1438.000000
3,13,OML2,RYK2,BD,226,0,2019-11-28 07:25:59.432470-05:00,2019-11-28 07:18:51.439817-05:00,1432.866667
4,14,RYK2,ISL2,BD,226,0,2019-11-28 07:27:50.445717-05:00,2019-11-28 07:21:43.450338-05:00,1433.883333
...,...,...,...,...,...,...,...,...,...
145,65,VMC2,VMC2,YUS,134,1,2019-11-28 09:34:43.741518-05:00,2019-11-28 09:32:43.700409-05:00,1437.983333
146,18,SPA1,SPA1,YUS,108,2,2019-11-28 09:28:51.367157-05:00,2019-11-28 09:28:43.901357-05:00,1439.866667
147,19,SGU1,SGU1,YUS,108,2,2019-11-28 09:30:51.366777-05:00,2019-11-28 09:30:43.686963-05:00,1439.866667
148,31,BLO1,BLO1,YUS,108,2,2019-11-28 09:48:51.109399-05:00,2019-11-28 09:47:51.101384-05:00,1438.983333


In [21]:
train_df[(train_df['cost'] < 1000)].sort_values(ascending = False, by = 'cost').reset_index(drop = True).head(50)

,sequence,dep_stn_char,arr_stn_char,subwayline,trainid,trip,dep_time,arr_time,cost
0,76,EGL1,PVL1,YUS,171,2,2019-11-28 09:38:43.538152-05:00,2019-11-28 11:39:43.577034-05:00,121.000000
1,75,DAV1,HWY1,YUS,171,2,2019-11-28 09:36:43.522855-05:00,2019-11-28 11:37:43.737187-05:00,121.000000
2,77,LAW1,YUN1,YUS,171,2,2019-11-28 09:43:08.683594440-05:00,2019-11-28 11:41:58.623198-05:00,118.816667
3,30,LAW1,DNP1,YUS,62,1,2019-11-28 09:32:02.632804500-05:00,2019-11-28 11:28:07.573015780-05:00,116.066667
4,28,YKM1,VMC2,YUS,126,0,2019-11-28 07:56:11.370453540-05:00,2019-11-28 09:46:43.646198-05:00,110.533333
5,4,FIN2,VMC1,YUS,56,0,2019-11-28 09:11:41.144825500-05:00,2019-11-28 11:00:43.339257-05:00,109.033333
6,7,NYC2,VMC2,YUS,51,1,2019-11-28 09:12:46.838302440-05:00,2019-11-28 10:56:43.656940-05:00,103.933333
7,41,VMC2,NYC1,YUS,169,1,2019-11-28 08:26:43.371523-05:00,2019-11-28 10:07:43.463721-05:00,101.000000
8,5,SHP1,VMC2,YUS,108,0,2019-11-28 07:13:40.269312640-05:00,2019-11-28 08:53:43.483579-05:00,100.050000
9,33,SHW1,GCN1,YUS,197,1,2019-11-28 09:57:35.915844500-05:00,2019-11-28 11:37:06.634520220-05:00,99.500000
